In [11]:
# Install the version of scikit-learn that was used to train the model
!pip install scikit-learn==1.2.2  # Replace with correct version if different

In [12]:
!pip install gradio

In [13]:
import requests
import pickle

def download_model(url, filename="gradient_boosting_model.pkl"):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes

        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Model downloaded successfully to {filename}")
        return filename

    except requests.exceptions.RequestException as e:
        print(f"Error downloading model: {e}")
        return None


model_url = "https://github.com/harshanasrimal/backpack-kaggle/raw/refs/heads/main/model/gradient_boosting_model.pkl"
downloaded_model_path = download_model(model_url)


if downloaded_model_path:
    try:
        with open(downloaded_model_path, "rb") as file:
            loaded_model = pickle.load(file)
        print("Model loaded successfully.")

    except (pickle.UnpicklingError, FileNotFoundError, EOFError) as e:
        print(f"Error loading model: {e}")


Model downloaded successfully to gradient_boosting_model.pkl
Model loaded successfully.


In [15]:
import gradio as gr
import joblib
import numpy as np

# Load the model
model = joblib.load("/content/gradient_boosting_model.pkl")

# Define function to preprocess inputs and predict
def predict_price(compartments, weight_capacity, laptop_compartment, waterproof, brand, material, size, style, color):
    # Convert categorical inputs into one-hot encoding (manual mapping)
    features = [
        int(compartments),
        1 if laptop_compartment else 0,
        1 if waterproof else 0,
        float(weight_capacity),

        # Brand
        1 if brand == "Jansport" else 0,
        1 if brand == "Nike" else 0,
        1 if brand == "Puma" else 0,
        1 if brand == "Under Armour" else 0,
        1 if brand == "Other" else 0,

        # Material
        1 if material == "Leather" else 0,
        1 if material == "Nylon" else 0,
        1 if material == "Polyester" else 0,
        1 if material == "Other" else 0,

        # Size
        1 if size == "Medium" else 0,
        1 if size == "Small" else 0,
        1 if size == "Other" else 0,

        # Style
        1 if style == "Messenger" else 0,
        1 if style == "Tote" else 0,
        1 if style == "Other" else 0,

        # Color
        1 if color == "Blue" else 0,
        1 if color == "Gray" else 0,
        1 if color == "Green" else 0,
        1 if color == "Pink" else 0,
        1 if color == "Red" else 0,
        1 if color == "Other" else 0,
    ]

    # Convert list to numpy array and reshape
    input_array = np.array(features).reshape(1, -1)

    # Make prediction
    predicted_price = model.predict(input_array)[0]

    return f"Predicted Price: ${predicted_price:.2f}"


# Define dropdown options
brand_options = ['Adidas', 'Jansport', 'Nike', 'Puma', 'Under Armour', 'Other']
material_options = ['Canvas', 'Leather', 'Nylon', 'Polyester', 'Other']
size_options = ['Large', 'Medium', 'Small', 'Other']
style_options = ['Backpack', 'Messenger', 'Tote', 'Other']
color_options = ['Black', 'Blue', 'Gray', 'Green', 'Pink', 'Red', 'Other']

# Create Gradio UI
interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label="Compartments", value=2, precision=0),
        gr.Number(label="Weight Capacity (kg)", value=10.0, precision=2),
        gr.Checkbox(label="Laptop Compartment"),
        gr.Checkbox(label="Waterproof"),
        gr.Dropdown(brand_options, label="Brand"),
        gr.Dropdown(material_options, label="Material"),
        gr.Dropdown(size_options, label="Size"),
        gr.Dropdown(style_options, label="Style"),
        gr.Dropdown(color_options, label="Color"),
    ],
    outputs=gr.Textbox(label="Predicted Price"),
    title="🎒 Backpack Price Prediction Tool by Harshana",
    description="Enter the details of the backpack and get an estimated price.",
    allow_flagging="never"
)

# Launch the app
interface.launch()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6858666c1515f32ea8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
